### EXAMEN NLP 

### IA SCHOOL : 2020-2021

| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Amadou lamarana      | DIALLO               |

## ETAPE 2: FEATURISATION DES COMMENTAIRES

#### Nous allons utiliser TF-IDF qui est une méthode de pondération souvent utilisée en recherche d'information et en particulier dans la fouille de textes. Cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document, relativement à une collection ou un corpus.


In [6]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import re
from unidecode import unidecode
import pickle
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
##Word2Vec
from gensim.models import Word2Vec
### CAMEMBERT
import torch
from transformers import CamembertTokenizer,CamembertModel

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [5]:
#!pip install transformers==3.3.1
#!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 3.5 MB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-macosx_10_9_x86_64.whl size=80635 sha256=a0668b1099e27a1d79eeaf519a8181edcbf2b7aa4f0f24fbca69f82b9dd35272
  Stored in directory: /Users/amadoudiallo/Library/Caches/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install gensim

In [ ]:
!pip install torch

#### 1) Recuperer le dataset des commentaires nettoyés après l'étape 1

In [7]:
commentaires_raw = pd.read_csv('commentaires_nettoyes_cinq_blessures_ok.csv')
commentaires_raw = commentaires_raw.sample(frac=1).reset_index(drop=True)

##### Dans le cadre de la création du modèle à la partie 3 nous allons scinder nos données en train et test

In [ ]:
y = commentaires_raw['label'].tolist()
X = commentaires_raw['commentaire'].tolist()
commentaires_raw[commentaires_raw.label==1]['label'].value_counts()

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

#### Nous allons effectuer la featurisation , c'est à dire transformer les mots en des valeurs numériques.. compréhensible par la machine 

### a) TF-IDF 

Cette méthode qui permet d'indentifier les mots du corpus très importants sur un document et pas sur les autres documents. 
Les features_names seront regroupé de 1 à 4 mots .. 

In [11]:
"""
Nous allons constitué le commentaire par un ensemble de groupe de mot de 2 à 4, afin de pouvoir comprendre le contexte 
"""
vectorizer = TfidfVectorizer(ngram_range=(1, 4))
train_tfid = vectorizer.fit_transform(X)
##
X_train_tfid, X_test_tfidf, y_train, y_test = train_test_split(train_tfid, y, test_size=0.2, random_state=43)

> Je vais maintenant utiliser le vectorizer sur le corpus prétraité de l'ensemble des commentaires pour extraire un vocabulaire et créer la matrice de caractéristiques.

In [ ]:
Mat=pd.DataFrame(train_tfid.toarray(),columns=vectorizer.get_feature_names())
Mat

Nous avons mettre nos commentaires sous forme de vecteurs numpy.. 

In [13]:
dic_vocabulary = vectorizer.vocabulary_

### le dictionnaire des vocabulaires avec les index

In [ ]:
dic_vocabulary

#### b) Word2Vec

<hr>
Je vais ajuster mon propre modèle Word2Vec sur le corpus de données d'entraînement avec gensim. Avant d'ajuster le modèle, le corpus doit être transformé en une liste de listes de n-grams.
<hr>
la taille cible des vecteurs de mots, j'utiliserai 200 ;
la fenêtre, ou la distance maximale entre le mot actuel et le mot prédit dans une phrase, j'utiliserai la longueur moyenne du texte dans le corpus ;
l'algorithme d'apprentissage, j'utiliserai le skip-grammes (sg=1) car en général il donne de meilleurs résultats.

In [15]:
liste = [elm.split() for elm in X] ## transformer les données d'entrainement en une liste de liste/

In [75]:
#### Creer le modèle word2Vec..

In [17]:
model_wv = Word2Vec(sentences=liste, vector_size=200, window=10, min_count=2, workers=8,sg=1)

In [22]:
### le nombre de corpus du document.. ( 500 commentaires)

In [ ]:
model_wv.corpus_count

#### on enregistre le modèle sous le nom "word2vec.model" que nous utiliserons pour la prochaine étape sur la classification

In [19]:
model_wv.save("word2vec.model")

In [ ]:
model_wv.wv.get_vector()

#### On peut afficher les textes avec leurs représentaions numériques

In [ ]:
"""
Boucle des 20 premiers words et leurs vecteurs..créés par Word2Vec
"""
i = 0
for elm in model_wv.wv.index_to_key:
    if i < 30:
        print(elm)
        print(model_wv.wv.similar_by_word(elm))
        print("----------")
        print(model_wv.wv.get_vector(elm, norm=True))
        
        #print(model_wv[elm])
        print('________')
        i= i+1

In [43]:
def nett(tex):
    """
    input: texte à nettoyer
    output: texte nettoyer de stop_words 
    """
    # Conserver tous les caracteres sauf les speciaux ($¨#([)) et le chiffres
    p='[a-zA-Zéâûèàùîôêç\-.!?:]{1,}'
    
    # stop words
    with open('mes_stop_words.txt') as f:
        s_w=f.read().split('\n')
    s_w.remove('fait')
    
    # Laisser un peu de place autour de chaque separateur de phrase
    tex=tex.replace('!',' ! ').replace('.',' . ').replace('?',' ? ').replace(':',' : ')
    
    final=[]
    for mot in re.findall(p,tex):
        if mot.lower() not in s_w :
            final.append(mot.lower())
    return ' '.join(final).strip()

#### Similarité des mots detecté par le modèle

In [ ]:
model_wv.wv.most_similar(nett('livre'))

#### Nous notons une mauvaise similiarité qui risque de causer préjudice au modèle avec Word2Vec

In [ ]:
model_wv.wv['lire']

#### c) Camembert (Le BERT français)

Nous allons réaliser la featurization en utilisant Camembert .. 

    1) récuperation du dataset et chargement du tokenizer de Camembert

In [ ]:
# Chargement du jeu de donnees
dataset = pd.read_csv("commentaires_nettoyes_cinq_blessures_ok.csv")
 
commentaires = dataset['commentaire'].values.tolist()
labels = dataset['label'].values.tolist()
 
# On charge l'objet "tokenizer"de camemBERT qui va servir a encoder

tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-base')

    2) Création du modèle 

In [ ]:
model_camb = CamembertModel.from_pretrained('camembert/camembert-base',output_hidden_states=True)

In [59]:
torch.save(model_camb.state_dict(), "./model_camb.pt")

#### on pourra le charger au plus tard avec la méthode load

In [ ]:
torch.load("model_camb.pt")

    3) Featuratisation des commentaires

#### La fonction batch_encode_plus encode un batch de données

In [61]:
## on encode les commentaires
encoded = tokenizer.batch_encode_plus(commentaires,
                                            add_special_tokens=True,
                                            padding=True,
                                            truncation=True,
                                            return_attention_mask = True,
                                            return_tensors = 'pt')

    4) On transforme la liste des labels en tenseur
 #### On transforme la liste des sentiments en tenseur

In [62]:
labels = torch.tensor(labels)

In [63]:
input_ids = tokenizer.encode("acheter ce livre est une pire perte",return_tensors='pt') 

In [ ]:
outputs = model_camb(input_ids)
outputs[2][-4:]

## Nous avons pu featurisé nos données. la suite est la partie 3 qui implémente les modèles avec les 3 méthodes 